## Consolidating Demand Data

This file reads power demand data from 2017 to 2022 and consolidates all the csv files into one. This file also includes code to generate simulated temeprature data which is then merged with the consolidated power demand data for further analysis.

In [1]:
import pandas as pd
import os

In [2]:
df_2017 = pd.read_csv('../data/demanddata_2017.csv')
df_2018 = pd.read_csv('../data/demanddata_2018.csv')
df_2019 = pd.read_csv('../data/demanddata_2019.csv')
df_2020 = pd.read_csv('../data/demanddata_2020.csv')
df_2021 = pd.read_csv('../data/demanddata_2021.csv')
df_2022 = pd.read_csv('../data/demanddata_2022.csv')

In [3]:
df_2017.head(1) 

,SETTLEMENT_DATE,SETTLEMENT_PERIOD,ND,TSD,ENGLAND_WALES_DEMAND,EMBEDDED_WIND_GENERATION,EMBEDDED_WIND_CAPACITY,EMBEDDED_SOLAR_GENERATION,EMBEDDED_SOLAR_CAPACITY,NON_BM_STOR,PUMP_STORAGE_PUMPING,IFA_FLOW,IFA2_FLOW,BRITNED_FLOW,MOYLE_FLOW,EAST_WEST_FLOW,NEMO_FLOW
0,01-JAN-2017,1,27316,28013,24789,1634,4871,0,11503,0,97,923,0,857,295,233,0


In [4]:
## df_2022 has different date format compared to other Dfs hence changing its format.
# Convert SETTLEMENT_DATE column to datetime
df_2022['SETTLEMENT_DATE'] = pd.to_datetime(df_2022['SETTLEMENT_DATE'], format='%Y-%m-%d', errors='coerce')

# Convert SETTLEMENT_DATE format to '01-JAN-2022'
df_2022['SETTLEMENT_DATE'] = df_2022['SETTLEMENT_DATE'].dt.strftime('%d-%b-%Y').str.upper()

### Combining all 6 years of data into 1 dataframe

In [5]:
demand_df = pd.concat([df_2017, df_2018, df_2019, df_2020, df_2021, df_2022])

### Resampling the demand data to 6 hour intervals since temperature data is avg of 6 hour temperatures. 

In [6]:
demand_df['SETTLEMENT_DATE'] = pd.to_datetime(demand_df['SETTLEMENT_DATE'], format='%d-%b-%Y', errors='coerce').combine_first(pd.to_datetime(demand_df['SETTLEMENT_DATE'], format='%d-%b-%Y', errors='coerce'))

# Resampling the demand data to 6 hour intervals since temperature data is avg of 6 hour temperatures.
demand_df['timestamp'] = demand_df['SETTLEMENT_DATE'] + pd.to_timedelta((demand_df['SETTLEMENT_PERIOD'] - 1) * 30, unit='m')
demand_df.set_index('timestamp', inplace=True)

In [7]:
demand_df

,SETTLEMENT_DATE,SETTLEMENT_PERIOD,ND,TSD,ENGLAND_WALES_DEMAND,EMBEDDED_WIND_GENERATION,EMBEDDED_WIND_CAPACITY,EMBEDDED_SOLAR_GENERATION,EMBEDDED_SOLAR_CAPACITY,NON_BM_STOR,PUMP_STORAGE_PUMPING,IFA_FLOW,IFA2_FLOW,BRITNED_FLOW,MOYLE_FLOW,EAST_WEST_FLOW,NEMO_FLOW,NSL_FLOW,ELECLINK_FLOW
timestamp,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,2017-01-01,1,27316,28013,24789,1634,4871,0,11503,0,97,923,0,857,295,233,0,NaN,NaN
2017-01-01 00:30:00,2017-01-01,2,27900,28539,25486,1630,4871,0,11503,0,39,997,0,874,297,287,0,NaN,NaN
2017-01-01 01:00:00,2017-01-01,3,27588,28279,25254,1660,4871,0,11503,0,91,997,0,818,297,287,0,NaN,NaN
2017-01-01 01:30:00,2017-01-01,4,26456,27819,24270,1689,4871,0,11503,0,763,997,0,822,297,287,0,NaN,NaN
2017-01-01 02:00:00,2017-01-01,5,25429,27565,23409,1736,4871,0,11503,0,1536,997,0,854,297,281,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 21:30:00,2022-12-31,44,24160,25634,21879,2592,6545,0,13861,0,184,1004,982,1002,-451,-239,999,1077.0,997.0
2022-12-31 22:00:00,2022-12-31,45,23223,24788,21040,2701,6545,0,13861,0,124,244,353,473,-451,-390,789,1077.0,999.0
2022-12-31 22:30:00,2022-12-31,46,22627,24365,20466,2810,6545,0,13861,0,173,114,302,422,-451,-514,772,1054.0,999.0


In [8]:
demand_6h = demand_df.resample('6H').mean()

In [9]:
demand_6h['SETTLEMENT_DATE'] = demand_6h.index.strftime('%d-%b-%Y')

In [10]:
last_col = demand_6h.pop(demand_6h.columns[-1])
demand_6h.insert(0, last_col.name, last_col)

In [11]:
demand_6h = demand_6h.reset_index(drop=False)

##### 2017 to 2019 and 2021 to 2022 = 365*5 + 366 for 2020 --> 2191 
##### 4 data points each day for 6 hour intervals so 2191*4 --> 8764

In [12]:
demand_6h.SETTLEMENT_DATE.describe()

count            8764
unique           2191
top       01-Jan-2017
freq                4
Name: SETTLEMENT_DATE, dtype: object

In [13]:
demand_6h.to_csv('../data/demanddata_2017_2022_6H.csv',index=False, float_format='%.2f')

In [14]:
demand_6h.head()

,timestamp,ELECLINK_FLOW,SETTLEMENT_DATE,SETTLEMENT_PERIOD,ND,TSD,ENGLAND_WALES_DEMAND,EMBEDDED_WIND_GENERATION,EMBEDDED_WIND_CAPACITY,EMBEDDED_SOLAR_GENERATION,EMBEDDED_SOLAR_CAPACITY,NON_BM_STOR,PUMP_STORAGE_PUMPING,IFA_FLOW,IFA2_FLOW,BRITNED_FLOW,MOYLE_FLOW,EAST_WEST_FLOW,NEMO_FLOW,NSL_FLOW
0,2017-01-01 00:00:00,NaN,01-Jan-2017,6.5,24507.416667,26336.583333,22437.000000,1776.000000,4871.0,0.0,11503.0,0.000000,1229.166667,720.666667,0.0,849.666667,271.250000,282.000000,0.0,NaN
1,2017-01-01 06:00:00,NaN,01-Jan-2017,18.5,26074.666667,27275.250000,24030.500000,1963.083333,4871.0,211.0,11503.0,0.000000,354.916667,86.333333,0.0,933.833333,226.250000,273.750000,0.0,NaN
2,2017-01-01 12:00:00,NaN,01-Jan-2017,30.5,34942.000000,35678.083333,32392.000000,2474.333333,4871.0,240.5,11503.0,0.000000,6.250000,981.750000,0.0,951.250000,-130.083333,243.083333,0.0,NaN
3,2017-01-01 18:00:00,NaN,01-Jan-2017,42.5,32169.666667,33286.083333,29460.416667,2102.833333,4871.0,0.0,11503.0,32.166667,11.083333,-167.833333,0.0,865.250000,-159.666667,271.250000,0.0,NaN
4,2017-01-02 00:00:00,NaN,02-Jan-2017,6.5,24598.666667,26419.250000,22276.416667,1507.333333,4871.0,0.0,11503.0,0.000000,1187.000000,381.666667,0.0,899.583333,48.916667,287.000000,0.0,NaN


## Reading Temp Data

In [15]:
import numpy as np
import random

In [16]:
df_temps = pd.read_csv('../data/UK_Temperatures.csv')

In [17]:
df_temps['observation_dtg_utc'] = pd.to_datetime(df_temps['observation_dtg_utc'])

In [18]:
df_temps = df_temps[df_temps['observation_dtg_utc'] < '2023-01-01 00:00:00']

## Combining both data sets

In [19]:
df_temps.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8759 entries, 0 to 8758
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   country              8759 non-null   object        
 1   observation_dtg_utc  8759 non-null   datetime64[ns]
 2   temp_c               8759 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 273.7+ KB


In [20]:
demand_6h.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8764 entries, 0 to 8763
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   timestamp                  8764 non-null   datetime64[ns]
 1   ELECLINK_FLOW              5844 non-null   float64       
 2   SETTLEMENT_DATE            8764 non-null   object        
 3   SETTLEMENT_PERIOD          8764 non-null   float64       
 4   ND                         8764 non-null   float64       
 5   TSD                        8764 non-null   float64       
 6   ENGLAND_WALES_DEMAND       8764 non-null   float64       
 7   EMBEDDED_WIND_GENERATION   8764 non-null   float64       
 8   EMBEDDED_WIND_CAPACITY     8764 non-null   float64       
 9   EMBEDDED_SOLAR_GENERATION  8764 non-null   float64       
 10  EMBEDDED_SOLAR_CAPACITY    8764 non-null   float64       
 11  NON_BM_STOR                8764 non-null   float64       
 12  PUMP_S

In [21]:
merged_df = pd.merge(demand_6h, df_temps, how = 'left',left_on='timestamp', right_on='observation_dtg_utc')

In [22]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8764 entries, 0 to 8763
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   timestamp                  8764 non-null   datetime64[ns]
 1   ELECLINK_FLOW              5844 non-null   float64       
 2   SETTLEMENT_DATE            8764 non-null   object        
 3   SETTLEMENT_PERIOD          8764 non-null   float64       
 4   ND                         8764 non-null   float64       
 5   TSD                        8764 non-null   float64       
 6   ENGLAND_WALES_DEMAND       8764 non-null   float64       
 7   EMBEDDED_WIND_GENERATION   8764 non-null   float64       
 8   EMBEDDED_WIND_CAPACITY     8764 non-null   float64       
 9   EMBEDDED_SOLAR_GENERATION  8764 non-null   float64       
 10  EMBEDDED_SOLAR_CAPACITY    8764 non-null   float64       
 11  NON_BM_STOR                8764 non-null   float64       
 12  PUMP_S

## Cleaning missing temp data

In [25]:
merged_df[merged_df.observation_dtg_utc.isnull()]

,timestamp,ELECLINK_FLOW,SETTLEMENT_DATE,SETTLEMENT_PERIOD,ND,TSD,ENGLAND_WALES_DEMAND,EMBEDDED_WIND_GENERATION,EMBEDDED_WIND_CAPACITY,EMBEDDED_SOLAR_GENERATION,...,IFA_FLOW,IFA2_FLOW,BRITNED_FLOW,MOYLE_FLOW,EAST_WEST_FLOW,NEMO_FLOW,NSL_FLOW,country,observation_dtg_utc,temp_c
6635,2021-07-17 18:00:00,0.000000,17-Jul-2021,42.5,25783.833333,26810.083333,23963.333333,962.416667,6527.0,676.166667,...,1913.333333,704.916667,912.666667,-148.083333,-337.583333,954.833333,-1.000000,NaN,NaT,25.5
6636,2021-07-18 00:00:00,0.000000,18-Jul-2021,6.5,19237.416667,20426.750000,17759.416667,651.833333,6527.0,13.333333,...,1443.000000,719.000000,729.750000,-333.416667,0.000000,721.833333,-1.000000,NaN,NaT,17.3
6640,2021-07-19 00:00:00,0.000000,19-Jul-2021,6.5,20834.416667,22696.500000,18908.583333,221.666667,6527.0,12.166667,...,1706.333333,749.166667,855.583333,-251.416667,-244.250000,844.333333,-1.000000,NaN,NaT,26.5
7049,2021-10-29 06:00:00,0.000000,29-Oct-2021,18.5,30294.583333,31715.166667,27287.333333,2185.583333,6527.0,716.750000,...,715.833333,607.666667,1025.500000,-441.333333,-465.500000,840.166667,692.916667,NaN,NaT,13.3
8528,2022-11-03 00:00:00,-1019.583333,03-Nov-2022,6.5,19477.250000,24198.500000,18700.833333,2744.333333,6545.0,0.000000,...,-1007.583333,-1012.000000,370.166667,-172.666667,-58.750000,175.916667,337.666667,NaN,NaT,12.7


#### Some of the timestamps have missing temperature data which can be cleaned up using interpolation method. 

In [24]:
merged_df['temp_c'].interpolate(method = 'nearest',inplace=True)

In [ ]:
merged_df[merged_df.timestamp >= '2021-07-17 06:00:00'].head(10)

In [ ]:
merged_df[merged_df.timestamp >= '2021-10-29 00:00:00'].head(3)

In [ ]:
merged_df[merged_df.timestamp >= '2022-11-02 18:00:00'].head(3)

## Creating clean data frame containing TSD and temps only

In [ ]:
final_clean_df = merged_df[['SETTLEMENT_DATE', 'timestamp','temp_c','TSD' ]].copy()

In [ ]:
final_clean_df=final_clean_df.rename(columns={'timestamp':'observation_dtg_utc'})

## Exporting the consolidated dataset

In [ ]:
final_clean_df.to_csv('../data/cleaned_temp_dmnd_2017_2022.csv', index =False, float_format='%.2f')